In [93]:
import requests
from bs4 import BeautifulSoup
from scrapingbee import ScrapingBeeClient
import math
import pandas as pd
from datetime import datetime, timedelta


In [102]:
def scrape_page(url):
    client = ScrapingBeeClient(api_key='7RGXHBFZ5Z598EJFDXJT5F7JMXY15EGTQ2FSDKX53QL8Y8G8975BPAY1LT0EH86WRFI3WOOK75ILGQYE')
    response = client.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    return soup

In [95]:
def scrape_one_page(job_links):
    data_list = []
    
    for job_list in job_links:
        single_page_scrape = scrape_page(job_list)
        #Job_Title
        job_title = single_page_scrape.h2
        if job_title is not None:
            job_title = job_title.get_text('h2').strip()
        else:
            job_title = " "

        #Job Url
        job_url = job_list

        #location
        location = single_page_scrape.find('h6', {'class': 'location inline-block m-r-30 m-t-0'})
        if location is not None:
            location_txt = location.text.strip()
        else:
            location_txt = " "
        
        # posted

        job_posted_time = None  # Initialize the variable with a default value
        posted_element = single_page_scrape.find('h6', class_='semi-bold date-posted')

        if posted_element is not None:
            job_posted_time = posted_element.text.strip()
        else:
            job_posted_time = ""

        # Define the date format
        date_format = "%Y-%m-%d"

        # Check if the time is given in days
        if "Posted days" in job_posted_time:
            # Subtract the corresponding number of days from the current date
            job_posted_date = datetime.now() - timedelta(days=int(job_posted_time.split()[0]))
            # Format the resulting date as a string
            job_posted_dates = job_posted_date.strftime(date_format)
        # Check if the time is given in hours
        elif "Posted hours" in job_posted_time:
            # Subtract the corresponding number of hours from the current date
            job_posted_date = datetime.now() - timedelta(hours=int(job_posted_time.split()[0]))
            # Format the resulting date as a string
            job_posted_dates = job_posted_date.strftime(date_format)
        # If the time is not given in days or hours, use the current date
        else:
            job_posted_dates = datetime.now()

        # Print the formatted date
        print(job_posted_dates)
        
        # description

        # job_description = single_page_scrape.find("div", class_="main-content")
        # description = job_description.find("div", class_="job-description hidden-text shadow")

        # if description is not None:
        #     description_txt = description.text.strip()
        # else:
        #     description_txt = " "

        job_description = single_page_scrape.find("div", class_="main-content")
        if job_description is not None:
            description = job_description.find("div", class_="job-description hidden-text shadow")
            if description is not None:
                description_txt = description.text.strip()
            else:
                description_txt = ""
        else:
            description_txt = ""

        data = {
            'job_title':job_title,
            'job_url':job_url,
            'location':location_txt,
            'posted':job_posted_dates,
            'description':description_txt

           
        }
        data_list.append(data)
    print(data_list)
    return data_list


In [104]:
job_to_scrape = 'Data Engineer'
l='Texas'
distance = 10
posted = "1d"
scrape = scrape_page(f"https://search.linkup.com/search/results/data-engineer-jobs-in-texas?pageNum=1&posted=7d")


roles = ["Data engineer", "Machine Learning"]

/tmp/ipykernel_423207/3251863856.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.content,'html.parser')


In [97]:
#To find page number
search_details = scrape.find('div', {'class': 'search-details'})
job_count_text = search_details.find('div').text.strip()
job_count = int(job_count_text.split()[-2].replace(',', ''))
page_number = math.ceil((job_count)/25)
print(job_count)
page_number


1381


56

In [100]:
job_links = []
i = 1

for role in roles:


    while i <= 3:
        print('Page number', i)
        scrape_result = scrape_page(f"https://search.linkup.com/search/results/{role}-jobs-in-texas?pageNum={i}&posted=7d")

        for link in scrape_result.find_all('a', class_='organic-link search-result-link'):
            x = "https://search.linkup.com"+link.get('href')
            job_links.append(x)

        # for j in range(len(job_links)):
        #     job_links[j] = 'https://search.linkup.com' + job_links[j]

        i = i + 1
        print(job_links)


Page number 1


/tmp/ipykernel_423207/3251863856.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.content,'html.parser')


[]
Page number 2
[]
Page number 3
[]


In [91]:
page_data = scrape_one_page(job_links)

[]


In [92]:
#Converting to dataframe
df = pd.DataFrame(page_data)
df.to_csv('linkup.csv',index=False)
df

""


In [26]:
# test_scrape = scrape_page(f"https://search.linkup.com/details/d08c18ab807b3efe1f8def1ffc1d3909")

In [27]:
# import datetime

# # Define the time elapsed since the job was posted
# job_posted_time = "2 days ago"

# # Define the date format
# date_format = "%Y-%m-%d"

# # Check if the time is given in days
# if "days" in job_posted_time:
#     # Subtract the corresponding number of days from the current date
#     job_posted_date = datetime.datetime.now() - datetime.timedelta(days=int(job_posted_time.split()[0]))
#     # Format the resulting date as a string
#     job_posted_dates = job_posted_date.strftime(date_format)
# # Check if the time is given in hours
# elif "hours" in job_posted_time:
#     # Subtract the corresponding number of hours from the current date
#     job_posted_date = datetime.datetime.now() - datetime.timedelta(hours=int(job_posted_time.split()[0]))
#     # Format the resulting date as a string
#     job_posted_dates = job_posted_date.strftime(date_format)
# # If the time is not given in days or hours, use the current date
# else:
#     job_posted_dates = datetime.datetime.now().strftime(date_format)

# # Print the formatted date
# print(job_posted_dates)